# Transolver
## 1. 模型简介
这是一个基于粒子模拟的MeshGraphNet示例。基于DeepMind
[Learning to Simulate](https://sites.google.com/view/learning-to-simulate/)
的工作。它演示了如何使用图神经网络（GNN）模拟拉格朗日流体、固体和可变形材料。
## 问题描述

**模型结构**<br>
该模型使用 MeshGraphNet 来捕捉流体系统的动态特性。系统被表示为一个图结构，其中顶点对应流体粒子，边表示它们之间的相互作用。该模型是自回归的，利用历史数据来预测未来状态。顶点的输入特征包括当前位置、速度、节点类型（例如流体、沙子、边界）以及历史速度。模型的输出为加速度，加速度定义为当前速度与下一时刻速度的差值。速度和加速度均由位置序列推导而来，并进行了标准高斯分布的归一化处理，以保证一致性。


## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>

### 2.2 软件环境检查


In [1]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("device =", device)
print("onescience version:", onescience.__version__)


============================ System Management Interface =============================
HCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      HCU%      Mode     
0       48.0C    30.0W      auto     300.0W     0%         0%        N/A      
1       49.0C    23.0W      auto     300.0W     0%         0%        N/A      
2       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
3       50.0C    23.0W      auto     300.0W     0%         0%        N/A      
=================================== End of SMI Log ===================================

torch version: 2.4.1
device = dtk
onescience version: 0.1.0


### 2.2 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 存在部分文件更新，请卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。
    
请在最顶层目录(包含setup.py文件的目录)，通过命令行，执行下述安装指令：
</div>

In [ ]:
!pip install -e .

## 3. 素材准备
### 3.1 数据集准备

我们使用[DeepMind的粒子物理数据集](https://sites.google.com/view/learning-to-simulate).
这些数据集中有些包含了粒子基础的流体飞溅和在盒子或立方体内弹跳的模拟，其他则使用了砂子或黏液等材料。共有17个数据集，其中部分数据集列举如下

| Datasets     | Num Particles | Num Time Steps |    dt    | Ground Truth Simulator |
|--------------|---------------|----------------|----------|------------------------|
| Water-3D     | 14k           | 800            | 5ms      | SPH                    |
| Water-2D     | 2k            | 1000           | 2.5ms    | MPM                    |
| WaterRamp    | 2.5k          | 600            | 2.5ms    | MPM                    |
| Sand         | 2k            | 320            | 2.5ms    | MPM                    |
| Goop         | 1.9k          | 400            | 2.5ms    | MPM                    |

此示例需要使用`tensorflow`库加载`.tfrecord`格式的数据集

In [ ]:
!pip install "tensorflow<=2.17.1"

要从DeepMind的仓库下载数据，请运行：

In [4]:
!cd ./raw_datasets
!bash download_dataset.sh Water ./data/

## 4 训练


### 4.1 单卡训练
此示例使用[Hydra](https://hydra.cc/docs/intro/)用于[实验](https://hydra.cc/docs/patterns/configuring_experiments/)
配置。Hydra提供了一种方便的方法来修改几乎任何实验参数，
例如数据集设置、模型配置和优化器选项，
通过命令行或配置文件。

要查看完整脚本选项，请运行以下命令：

In [ ]:
!python train.py --help

如果遇到 Hydra 配置问题，可能会收到不太有帮助的错误信息。此时，可以设置环境变量 `HYDRA_FULL_ERROR=1`，以获取更详细的错误信息。以使用Goop数据集训练模型为例，请运行：

In [ ]:
!HYDRA_FULL_ERROR=1 python train.py +experiment=goop data.data_dir=./data/Goop resume_dir=./model/Goop

### 4.2 多卡训练
运行`mpirun -np <num_GPUs> --allow-run-as-root python main.py`进行训练

主要参数介绍，其他参数参见单卡介绍：
- num_GPUs：卡的数量

In [ ]:
!mpirun -np 4 --allow-run-as-root python train.py ...

### 4.3 跨节点多卡训练

torchrun启动多节点多卡训练：`torchrun --standalone --nnodes=<num_nodes> --nproc_per_node=<num_GPUs> main.py`
主要参数介绍，其他详细可参考torchrun参数：
- num_nodes： 节点数量
- num_GPUs：卡的数量

In [ ]:
!torchrun --standalone --nnodes=1 --nproc_per_node=4 train.py ...

## 5 推理

### 5.1 推理
推理脚本 `inference.py` 也支持 Hydra 配置，确保训练和推理过程的一致性。

模型训练完成后，运行以下命令：
```bash
python inference.py +experiment=water \
    data.data_dir=./data/Water \
    data.test.num_sequences=4 \
    resume_dir=./model /water \
    output=./result/water/inference
```
`num_sequences`参数表示从测试集中选取的测例个数

`resume_dir` 参数指定模型检查点的位置。

`output`参数指定推理结果的保存位置，测试数据集的预测结果会保存为动画 `.gif` 文件，存放在`output`路径下的 `animations` 文件夹中。

脚本还会生成一个 `error.png` 文件，用于展示展开误差的可视化。

In [ ]:
python inference.py +experiment=water  data.data_dir=./data/Water data.test.num_sequences=4  resume_dir=./model/water  output=./result/water/inference